In [2]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import io, json, gspread, gspread_dataframe, math, time, requests, os
import pandas as pd
from tqdm import tqdm
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.discovery import Resource
import numpy as np
from google_auth_oauthlib.flow import InstalledAppFlow
import settings_notebook as settings

In [4]:
dim_area_column = [
    "area_code",
    "area_name",
    "area_type",
    "region_code",
    "region_name",
    "income_level_code",
    "income_level_name"
]

dim_indicator_column = [
    "indicator_code",
    "indicator_name",
    "category_id",
    "category_name",
    "sub_category_id",
    "sub_category_name",
    "indicator_area_id",
    "indicator_area_name",
    "unit",
]

dim_year_column = [
    "year",
    "note"
]

fact_digital_equity = [
    "dim_year_id",
    "dim_indicator_id",
    "dim_area_id",
    "value",
    "grade_index"
]

default_null_value = "--EMPTY VALUE DATA--"

In [15]:
def get_creds(credentials_file: str, token_file: str, scope:list[str]) -> Credentials:
    creds = None
    if os.path.exists(token_file):
        creds = Credentials.from_authorized_user_file(token_file, scope)
    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file(
            credentials_file, scope)
        creds = flow.run_local_server(port=0)
        with open(token_file, "w") as token:
            token.write(creds.to_json())   
    return creds

def get_all_cleaned_data(creds, cleaned_data_folder_id:str):
    def list_files_in_folder(service:Resource, folder_id:str):
        
        try:
          query = f"'{folder_id}' in parents"
          
          results = (
              service.files()
              .list(q=query, fields="nextPageToken, files(id, name, mimeType)")
              .execute()
          )
          items = results.get("files", [])
          return items

        except HttpError as error:
          print(f"An error occurred while listing files: {error}")
          return None
      
    converted_column = [
                        "Province",
                        "Indicator ID",
                        "2018", "2019", "2020", "2021", "2022", "2023"]
    
    try:
        service = build("drive", "v3", credentials=creds)
    except HttpError as error:
        print(f"An error occurred: {error}")
    
    file_items = list_files_in_folder(service, cleaned_data_folder_id)
    df_list:list[pd.DataFrame] = list()
    sps_client = gspread.authorize(creds)
    for file in tqdm(file_items, desc="Processing Google Sheets files"):
        if file['mimeType'] == "application/vnd.google-apps.spreadsheet" and file['name'].endswith("cleaned"):
            sheet_id:str = file['id']
            sheet_name:str = file['name']
            
            try:
                spreadsheet = sps_client.open_by_key(sheet_id)
                worksheet = spreadsheet.worksheet("main")
                df = gspread_dataframe.get_as_dataframe(worksheet=worksheet)   
            except gspread.exceptions.APIError as e:
                print(f"Error: Quota Exceeded | Retrying in 65 seconds . . .")
                time.sleep(65)
                spreadsheet = sps_client.open_by_key(sheet_id)
                worksheet = spreadsheet.worksheet("main")
                df = gspread_dataframe.get_as_dataframe(worksheet=worksheet)
            except Exception as e:
                raise Exception(f"error while getting data from {sheet_name} : {e}")
            
            df.columns = list(map(lambda x: str(x), df.columns))
            df = df[converted_column]
            if list(df.columns) != converted_column:
                raise Exception(f"Invalid Column Structure from {sheet_name} with {list(df.columns)}")
            df_list.append(df)
    print(f"\n{len(df_list)} DATAFRAME(S) COLLECTED FROM GOOGLE SHEETS")
    return df_list

def concatenate_cleaned_data(df_list:list[pd.DataFrame]) -> pd.DataFrame:
    concatenated_df = pd.concat(df_list, ignore_index=True)
    
    print(f"CONCATENATED DATAFRAME SHAPE: {concatenated_df.shape}")
    print(f"{len(concatenated_df.Province.unique())} UNIQUE PROVINCE DETECTED")
    print(f"{len(concatenated_df['Indicator ID'].unique())} UNIQUE INDICATOR DETECTED")
    print(f"{len(concatenated_df[~concatenated_df['2018'].isnull()])} NON-EMPTY VALUE DATA FROM 2018")
    print(f"{len(concatenated_df[~concatenated_df['2019'].isnull()])} NON-EMPTY VALUE DATA FROM 2019")
    print(f"{len(concatenated_df[~concatenated_df['2020'].isnull()])} NON-EMPTY VALUE DATA FROM 2020")
    print(f"{len(concatenated_df[~concatenated_df['2021'].isnull()])} NON-EMPTY VALUE DATA FROM 2021")
    print(f"{len(concatenated_df[~concatenated_df['2022'].isnull()])} NON-EMPTY VALUE DATA FROM 2022")
    print(f"{len(concatenated_df[~concatenated_df['2023'].isnull()])} NON-EMPTY VALUE DATA FROM 2023")
    
    return concatenated_df

def melt_cleaned_data(concatenated_df:pd.DataFrame) -> pd.DataFrame:
    melted_df = pd.melt(concatenated_df, 
                        id_vars=["Province", "Indicator ID"], 
                        value_vars=["2018", "2019", "2020", "2021", "2022", "2023"],
                        var_name="Year",
                        value_name="Value")
    melted_df = melted_df.sort_values(by=["Province", "Indicator ID", "Year"])
    melted_df = melted_df.rename(columns={"Province":"Area",
                                          "Indicator ID":"Indicator Code"})
    
    print(f"MELTING DATA BY YEAR FROM {len(concatenated_df)} DATA to {len(melted_df)} DATA")
    print(f"MELTED DATAFRAME SHAPE: {melted_df.shape}")
    
    return melted_df

def get_master_data(creds, master_worksheet:str): # (ID, GID)
  
    client = gspread.authorize(creds)
    
    master_area_client = client.open_by_key(settings.MASTER_AREA_SPSID)
    master_area_worksheet = master_area_client.worksheet(master_worksheet)
    try:
        master_area_df = gspread_dataframe.get_as_dataframe(worksheet=master_area_worksheet, )
    except HttpError:
        print("Quota Exceeded | Wait a minute . . .")
        master_area_df = gspread_dataframe.get_as_dataframe(worksheet=master_area_worksheet, )
    master_area_df["ID"] = master_area_df["ID"].astype("int64").astype(str).apply(lambda x : x.zfill(2))
    
    master_inc_province_client = client.open_by_key(settings.MASTER_INCOME_PROVINCE_SPSID)
    master_inc_province_worksheet = master_inc_province_client.worksheet(master_worksheet)
    try:
        master_inc_province_df = gspread_dataframe.get_as_dataframe(worksheet=master_inc_province_worksheet)
    except HttpError:
        print("Quota Exceeded | Wait a minute . . .")
        master_inc_province_df = gspread_dataframe.get_as_dataframe(worksheet=master_inc_province_worksheet)
        
    master_year_client = client.open_by_key(settings.MASTER_YEAR_SPSID)
    master_year_worksheet = master_year_client.worksheet(master_worksheet)
    master_year_df = gspread_dataframe.get_as_dataframe(worksheet=master_year_worksheet)
    master_year_df.Year = master_year_df.Year.astype(int).astype(str)
    
    master_indicator_client = client.open_by_key(settings.MASTER_INDICATOR_SPSID)
    master_indicator_worksheet = master_indicator_client.worksheet(master_worksheet)
    master_indicator_df = gspread_dataframe.get_as_dataframe(worksheet=master_indicator_worksheet, )
    
    print(f"ROWS OF AREA MASTER DATA : {len(master_area_df)}")
    print(f"ROWS OF PROVINCE INCOME MASTER DATA : {len(master_inc_province_df)}")
    print(f"ROWS OF YEAR MASTER DATA : {len(master_year_df)}")
    print(f"ROWS OF INDICATOR MASTER DATA : {len(master_indicator_df)}")
    
    return master_area_df, master_inc_province_df, master_year_df, master_indicator_df
def cross_merge_master(master_area:pd.DataFrame,
                       master_year:pd.DataFrame, 
                       master_indicator:pd.DataFrame, 
                       column_to_rename:dict):

    province = master_area.drop_duplicates(subset=["ID", "AREA_NAME"]).reset_index(drop=True)
    indicator = master_indicator.drop_duplicates(subset=["Indicator_Code", "Unit"]).reset_index(drop=True)
    year = master_year.astype({"Year":str}).drop_duplicates(subset=["Year"])
    
    print(f"FOUND {len(province)} PROVINCE DATA")
    print(f"FOUND {len(indicator)} INDICATOR DATA")
    print(f"FOUND {len(year)} YEAR DATA")
    
    return province.merge(indicator, how='cross').merge(year, how='cross').rename(columns=column_to_rename)

def validate_data_1(cross_merged_df:pd.DataFrame):
    if cross_merged_df[cross_merged_df[["Area", "Indicator Code", "Year"]].duplicated()].shape[0] > 0:
        raise Exception("There('re/'s) Duplicate Data in MASTER DATA")
    

def validate_data_2(cross_merged_df:pd.DataFrame, melted_df:pd.DataFrame):
    print("Ensuring cleaned data is a subset of master data . . .")
    ensure_province:bool = set(melted_df.Area).issubset(cross_merged_df.Area) 
    ensure_year:bool = set(melted_df.Year).issubset(cross_merged_df.Year) 
    ensure_indicator:bool = set(melted_df["Indicator Code"]).issubset(cross_merged_df["Indicator Code"])
    if not (ensure_province and ensure_indicator and ensure_year):
        raise Exception("There('re/'s) Invalid Data from CLEANED DATA")
    
def left_outer_merge_to_master(cross_merged_df:pd.DataFrame, melted_df:pd.DataFrame):
    result = pd.merge(cross_merged_df,
                      melted_df, 
                      on=["Area", "Indicator Code", "Year"], how="left", )

    return result

def convert_value_dataframe(cleaned_data:pd.DataFrame):
    def preprocess_value(unit, value):
        if isinstance(value, float) and math.isnan(value) : return np.nan
        if value == "-" : return np.nan
        match unit:
            case "%":
                return str(value).replace(",", ".")
            case "Average":
                return str(value).replace(",", ".")
            case "Count":
                try:
                    return int(value)
                except ValueError:
                    return str(value).replace(".", "").replace(",", "")
            case "Rupiah":
                return str(value).replace("", "")
            case _:
                raise Exception(f"Invalid Unit '{unit}'")
    
    
    converted_result = cleaned_data.copy(deep=True)
    converted_result.Value = converted_result.apply(lambda row: preprocess_value(row.Unit, row.Value), axis=1).astype(float)
    
    return converted_result 


def write_data_to_sps(creds, sps_id:str, worksheet_name:str, df:pd.DataFrame):
    client = gspread.authorize(creds)

    spreadsheet = client.open_by_key(sps_id)
    worksheet = spreadsheet.worksheet(worksheet_name)
    worksheet.clear()
    
    area_as_a_text_format = {
        "numberFormat": {
            "type": "TEXT"
        }
    }
    num_rows = len(df) + 1
    column_range = f'A2:A{num_rows}'
    worksheet.format(column_range, area_as_a_text_format)
    
    gspread_dataframe.set_with_dataframe(worksheet=worksheet, dataframe=df)
    border_style = {
        "style": "SOLID",
        "width": 1,
    }

    formatting = {
        "borders": {
          "top": border_style,
          "bottom": border_style,
          "left": border_style,
          "right": border_style
        }
    }
    num_rows = len(df) + 1
    num_cols = len(df.columns)
    range_to_border = f"A1:{gspread.utils.rowcol_to_a1(num_rows, num_cols)}"
    worksheet.format(range_to_border, formatting)
    
    bold_format = {
        "textFormat":{
            "bold":True
        }
    }
    range_to_bold = f"A1:{gspread.utils.rowcol_to_a1(1, len(df.columns))}"
    worksheet.format(range_to_bold, bold_format)
    
    requests = [
        {
            "autoResizeDimensions": {
                "dimensions": {
                    "sheetId": worksheet.id, # The numeric ID of the worksheet
                    "dimension": "COLUMNS",
                    "startIndex": 0,
                    "endIndex": len(df.columns)
                }
            }
        }
    ]

    # Send the batch update request to the spreadsheet
    spreadsheet.batch_update(body={'requests': requests})
    
    return df

def get_dataframe_from_sheet(creds, sps_id:str, worksheet_name:str):
    client = gspread.authorize(creds)

    spreadsheet = client.open_by_key(sps_id)
    worksheet = spreadsheet.worksheet(worksheet_name)
    df = gspread_dataframe.get_as_dataframe(worksheet=worksheet)
    
    return df

def handle_dim_year(creds:Credentials,
                    dim_year:pd.DataFrame,
                    master_year_df:pd.DataFrame,
                    default_null_value:str) -> pd.DataFrame:
    dim_year[["year"]] = dim_year[["year"]].astype(int).astype(str)
    
    # Implementation of SCD TYPE 1
    rename_source_year = master_year_df.fillna(default_null_value).rename(columns={"Year":"year", "Notes":"note"})
    drop_duplicates_source_year = rename_source_year.drop_duplicates(subset=["year"], keep="last").set_index("year")
    dim_year_buskey_as_index = dim_year.set_index("year")
    dim_year_buskey_as_index.update(drop_duplicates_source_year)
    
    # ADD NEW DATA
    new_year = drop_duplicates_source_year[~drop_duplicates_source_year.index.isin(dim_year_buskey_as_index.index)]
    concatenated_dim_year = pd.concat([dim_year_buskey_as_index, new_year])
    
    # WRITE TO SPS
    final_dim_year = concatenated_dim_year.reset_index()
    final_dim_year.insert(0, 'id', range(1, len(final_dim_year)+1))
    write_data_to_sps(creds, settings.WAREHOUSE_DATA_SPS_ID, "dim_year", final_dim_year)
    return final_dim_year

def handle_dim_area(creds:Credentials, 
                        dim_area:pd.DataFrame,
                        master_area_df:pd.DataFrame,
                        master_inc_province_df:pd.DataFrame,
                        default_null_value:str) -> pd.DataFrame:
    
    # HANDLE dim_area
    if not set(master_inc_province_df["Provinsi"]).issubset(set(master_area_df["AREA_NAME"])):
        raise Exception("Invalid Province Name Found")

    dim_area["area_code"] = dim_area["area_code"].astype(int).astype(str).apply(lambda dt: dt.zfill(2))

    # RENAMED MASTER
    renamed_master_inc =  master_inc_province_df.rename(columns={"Provinsi":"area_name",
                                                        "Tingkat_Pendapatan":"income_level_name",
                                                        "ID_Pendapatan":"income_level_code"})
    renamed_master_area = master_area_df.rename(columns={ "ID":"area_code",
                                                "AREA_NAME":"area_name",
                                                "AREA_TYPE":"area_type",
                                                "REGION_GROUP":"region_name",
                                                "ID_REGION":"region_code"
                                                })

    # MERGE AREA
    merged_area =  renamed_master_area.merge(renamed_master_inc,on=["area_name"], how="left")[dim_area.columns]

    # Implementation of SCD TYPE 1
    rename_source_area = merged_area.fillna(default_null_value)
    drop_duplicates_source_area = rename_source_area.drop_duplicates(subset=["area_code"], keep="last").set_index("area_code")
    dim_area_buskey_as_index = dim_area.set_index("area_code")
    dim_area_buskey_as_index.update(drop_duplicates_source_area)

    # ADD NEW DATA
    new_location = drop_duplicates_source_area[~drop_duplicates_source_area.index.isin(dim_area_buskey_as_index.index)]
    concatenated_dim_area = pd.concat([dim_area_buskey_as_index, new_location])

    # WRITE TO SPS
    final_dim_area = concatenated_dim_area.reset_index()
    final_dim_area.insert(0, 'id', range(1, len(final_dim_area)+1))
    write_data_to_sps(creds, settings.WAREHOUSE_DATA_SPS_ID, "dim_area", final_dim_area)
    
    return final_dim_area

def handle_dim_indicator(creds:Credentials, 
                        dim_indicator:pd.DataFrame,
                        master_indicator_df:pd.DataFrame,
                        default_null_value:str) -> pd.DataFrame:
    
    # IMPLEMENTATION OF SCD TYPE 1
    rename_source_indicator = master_indicator_df.fillna(default_null_value).rename(columns={
                                                                                            "Indicator_Code":"indicator_code",
                                                                                            "Indicator_Name":"indicator_name",
                                                                                            "Category_ID":"category_id",
                                                                                            "Category":"category_name",
                                                                                            "Sub_Category_ID":"sub_category_id",
                                                                                            "Sub_Category":"sub_category_name",
                                                                                            "Area_ID":"indicator_area_id",
                                                                                            "Area_Type":"indicator_area_name",
                                                                                            "Unit":"unit"
                                                                                            })[dim_indicator.columns]
    drop_duplicates_source_indicator = rename_source_indicator.drop_duplicates(subset=["indicator_code"], keep="last").set_index("indicator_code")
    dim_indicator_buskey_as_index = dim_indicator.set_index("indicator_code")
    dim_indicator_buskey_as_index.update(drop_duplicates_source_indicator)


    # ADD NEW DATA
    new_indicator = drop_duplicates_source_indicator[~drop_duplicates_source_indicator.index.isin(dim_indicator_buskey_as_index.index)]
    concatenated_dim_indicator = pd.concat([dim_indicator_buskey_as_index, new_indicator])

    # WRITE TO SPS
    final_dim_indicator = concatenated_dim_indicator.reset_index()
    final_dim_indicator.insert(0, 'id', range(1, len(final_dim_indicator)+1))
    write_data_to_sps(creds, settings.WAREHOUSE_DATA_SPS_ID, "dim_indicator", final_dim_indicator)
    
    return final_dim_indicator

def handle_fact_table(creds:Credentials, 
                      warehouse_sps_id:str, 
                      worksheet_name:str, 
                      data_source:pd.DataFrame, # ["Indicator Code", "Area Code", "Year", "Value", "q_position"]
                      final_dim_year:pd.DataFrame, # final dim
                      final_dim_area:pd.DataFrame, # final dim
                      final_dim_indicator:pd.DataFrame # final dim
                      ) -> pd.DataFrame:
    data_source.columns = ["indicator_code", "area_code", "year", "value", "grade_index"]
    enriched_df = data_source.merge(final_dim_year[["id", "year"]], how="inner", on="year").rename(columns={"id":"dim_year_id"})
    enriched_df = enriched_df.merge(final_dim_area[["id", "area_code"]], how="inner", on="area_code").rename(columns={"id":"dim_area_id"})
    enriched_df = enriched_df.merge(final_dim_indicator[["id", 
                                                        "indicator_code"]], 
                                    how="inner", 
                                    on="indicator_code").rename(columns={"id":"dim_indicator_id"})
    final_fact_digital_equity = enriched_df[enriched_df.value.notnull()][fact_digital_equity]
    write_data_to_sps(creds, warehouse_sps_id, worksheet_name, final_fact_digital_equity)
    return final_fact_digital_equity

## Get Credentials for accessing Google Drive API

In [13]:
creds = get_creds("credentials.json", "token.json", settings.SCOPES)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=846575203738-q00v5rmc6jlmtvkt707ko5pnabj4bak6.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A52085%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=0aiZm5zRSSZ6MDzEa5cLjOMmoLApOP&access_type=offline


## Cleaned Data -> Converted Data

In [16]:
cleaned_data:list[pd.DataFrame] = get_all_cleaned_data(creds, settings.CLEANED_DATA_DRIVE_FOLDER)

Processing Google Sheets files:  33%|███▎      | 31/94 [00:55<01:54,  1.81s/it]

Error: Quota Exceeded | Retrying in 65 seconds . . .


Processing Google Sheets files:  64%|██████▍   | 60/94 [02:57<01:09,  2.03s/it]

Error: Quota Exceeded | Retrying in 65 seconds . . .


Processing Google Sheets files:  93%|█████████▎| 87/94 [04:54<00:13,  1.92s/it]

Error: Quota Exceeded | Retrying in 65 seconds . . .


Processing Google Sheets files: 100%|██████████| 94/94 [06:14<00:00,  3.99s/it]


94 DATAFRAME(S) COLLECTED FROM GOOGLE SHEETS


In [8]:
concatenated_df:pd.DataFrame = concatenate_cleaned_data(cleaned_data)

CONCATENATED DATAFRAME SHAPE: (3278, 8)
35 UNIQUE PROVINCE DETECTED
94 UNIQUE INDICATOR DETECTED
203 NON-EMPTY VALUE DATA FROM 2018
404 NON-EMPTY VALUE DATA FROM 2019
2510 NON-EMPTY VALUE DATA FROM 2020
2556 NON-EMPTY VALUE DATA FROM 2021
2048 NON-EMPTY VALUE DATA FROM 2022
2542 NON-EMPTY VALUE DATA FROM 2023


In [9]:
melted_df:pd.DataFrame = melt_cleaned_data(concatenated_df)

MELTING DATA BY YEAR FROM 3278 DATA to 19668 DATA
MELTED DATAFRAME SHAPE: (19668, 4)


In [10]:
melted_df[melted_df.Value.notnull()]

,Area,Indicator Code,Year,Value
13078,Aceh,NUMVILL.RURALONLY.2021,2021,5765.0
13044,Aceh,NUMVILL.URBANONLY.2021,2021,749.0
9626,Aceh,OWNCOMP.PROV,2020,"17,31"
12904,Aceh,OWNCOMP.PROV,2021,"15,62"
16182,Aceh,OWNCOMP.PROV,2022,"13,70"
...,...,...,...,...
19146,Sumatera Utara,USECOMP.RUR,2023,"6,33"
9347,Sumatera Utara,USECOMP.URB,2020,"17,99"
12625,Sumatera Utara,USECOMP.URB,2021,"14,37"
15903,Sumatera Utara,USECOMP.URB,2022,"14,51"


In [11]:
master_area_df, master_inc_province_df, master_year_df, master_indicator_df = get_master_data(creds, settings.MASTER_WORKSHEET)

ROWS OF AREA MASTER DATA : 35
ROWS OF PROVINCE INCOME MASTER DATA : 34
ROWS OF YEAR MASTER DATA : 6
ROWS OF INDICATOR MASTER DATA : 199


In [12]:
column_to_rename = {
        "ID":"Area Code",
        "AREA_NAME":"Area",
        "Indicator_Code":"Indicator Code",
        "Indicator_Name":"Indicator Name",
        "Unit":"Unit"
    }
cross_merged_df = cross_merge_master(master_area_df, master_year_df, master_indicator_df, column_to_rename)

# random.seed(199) 
# def generate_value(row):
#     if row['Unit'] == 'Rupiah':
#         return random.randint(100_000, 5_000_000)
#     elif row['Unit'] == '%':
#         return round(random.uniform(50, 100), 2)
#     elif row['Unit'] == 'Average':
#         return round(random.uniform(1, 3), 2)
#     elif row['Unit'] == 'Count':
#         return random.randint(100, 5000)
    
#     raise ValueError(f"Unknown unit: {row['Unit']}")
# cross_merged_master['Value'] = cross_merged_df.apply(generate_value, axis=1)

FOUND 35 PROVINCE DATA
FOUND 199 INDICATOR DATA
FOUND 6 YEAR DATA


In [13]:
validate_data_1(cross_merged_df)

In [14]:
validate_data_2(cross_merged_df, melted_df)

Ensuring cleaned data is a subset of master data . . .


In [15]:

result = left_outer_merge_to_master(cross_merged_df, melted_df)

In [16]:
converted_result = convert_value_dataframe(result)

In [17]:
converted_result[converted_result.Value.notnull()]

,Area Code,Area,AREA_TYPE,REGION_GROUP,ID_REGION,Indicator Name,Indicator Code,Category,Category_ID,Sub_Category,Sub_Category_ID,Area_Type,Area_ID,Unit,threshold_grade_a,threshold_grade_b,threshold_grade_c,Year,Notes,Value
26,11,Aceh,Province,Sumatra,SM,Average Number of Active Mobile Phone Numbers ...,TEL.MOB.PROV.ACTNUM.AVG,Technology Ownership and Use,TECH_USE,Telephone,TEL,All,ALL,Average,2.728,2.8460,2.954,2020,NaN,2.70
27,11,Aceh,Province,Sumatra,SM,Average Number of Active Mobile Phone Numbers ...,TEL.MOB.PROV.ACTNUM.AVG,Technology Ownership and Use,TECH_USE,Telephone,TEL,All,ALL,Average,2.728,2.8460,2.954,2021,NaN,2.71
28,11,Aceh,Province,Sumatra,SM,Average Number of Active Mobile Phone Numbers ...,TEL.MOB.PROV.ACTNUM.AVG,Technology Ownership and Use,TECH_USE,Telephone,TEL,All,ALL,Average,2.728,2.8460,2.954,2022,NaN,2.70
29,11,Aceh,Province,Sumatra,SM,Average Number of Active Mobile Phone Numbers ...,TEL.MOB.PROV.ACTNUM.AVG,Technology Ownership and Use,TECH_USE,Telephone,TEL,All,ALL,Average,2.728,2.8460,2.954,2023,NaN,3.03
32,11,Aceh,Province,Sumatra,SM,Average Number of Household Members Owning Mob...,TEL.MOB.PROV.MEMBERS.AVG,Technology Ownership and Use,TECH_USE,Telephone,TEL,All,ALL,Average,2.410,2.5100,2.600,2020,NaN,2.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41783,00,Indonesia,Nasional,Indonesia,IDN,Percentage of Households Owning/Controlling Mo...,TEL.OWN.MOB.URB.RES.FREE,Technology Ownership and Use,TECH_USE,Telephone,TEL,Urban,URB,%,10.808,11.8130,13.134,2023,NaN,13.18
41786,00,Indonesia,Nasional,Indonesia,IDN,Percentage of Households Owning/Controlling Mo...,TEL.OWN.MOB.URB.RES.PRIV,Technology Ownership and Use,TECH_USE,Telephone,TEL,Urban,URB,%,69.820,73.8205,77.630,2020,NaN,71.30
41787,00,Indonesia,Nasional,Indonesia,IDN,Percentage of Households Owning/Controlling Mo...,TEL.OWN.MOB.URB.RES.PRIV,Technology Ownership and Use,TECH_USE,Telephone,TEL,Urban,URB,%,69.820,73.8205,77.630,2021,NaN,72.89
41788,00,Indonesia,Nasional,Indonesia,IDN,Percentage of Households Owning/Controlling Mo...,TEL.OWN.MOB.URB.RES.PRIV,Technology Ownership and Use,TECH_USE,Telephone,TEL,Urban,URB,%,69.820,73.8205,77.630,2022,NaN,73.20


In [37]:
# Set Threshold
non_null_df = converted_result[converted_result.Value.notnull()]
province_only_df = non_null_df[non_null_df["Area Code"] != "00"].sort_values(by=["Indicator Code","Area Code", "Year"])
get_unique = province_only_df[["Indicator Code", "Value", "Year", "Area Code"]].groupby(by=["Indicator Code"]).apply(lambda x: x)
get_unique = get_unique[["Indicator Code", "Area Code", "Year", "Value" ]].reset_index(drop=True)
# get_unique = province_only_df.groupby(by=["Indicator Code"]).apply(lambda x: x.drop_duplicates(subset=['Value'])).reset_index()
get_unique["length"] = get_unique.groupby(by=["Indicator Code"])["Value"].transform(len)
get_unique["rank"] = get_unique.groupby(by=["Indicator Code"])["Value"].rank(method='min', ascending=True)
get_unique = get_unique.astype({"rank":"int64", "length":"int64"})
get_unique["q_position"] = ((get_unique["rank"] - 1) / (get_unique["length"] - 1)) * 100
get_threshold_df = get_unique.groupby(by=["Indicator Code"])["Value"].quantile([0.4, 0.55, 0.7]).to_frame().reset_index()
threshold_pivot_df = get_threshold_df.pivot(index=["Indicator Code"], columns="level_1", values="Value").reset_index()
threshold_final_df = threshold_pivot_df.rename(columns={0.7:"threshold_grade_a", 0.55:"threshold_grade_b", 0.4:"threshold_grade_c"})
merged_threshold_final_df = get_unique.merge(threshold_final_df, how="left", on="Indicator Code")[["Indicator Code",
                                                                                            "Area Code",
                                                                                            "Year",
                                                                                            "q_position",
                                                                                            "threshold_grade_a",
                                                                                            "threshold_grade_b",
                                                                                            "threshold_grade_c"]]
data_with_threshold = converted_result.merge(merged_threshold_final_df, how="left", on=["Indicator Code", "Area Code", "Year"])
#data_with_threshold
data_with_threshold

/tmp/ipykernel_17926/2449658651.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  get_unique = province_only_df[["Indicator Code", "Value", "Year", "Area Code"]].groupby(by=["Indicator Code"]).apply(lambda x: x)


,Area Code,Area,AREA_TYPE,REGION_GROUP,ID_REGION,Indicator Name,Indicator Code,Category,Category_ID,Sub_Category,...,threshold_grade_a_x,threshold_grade_b_x,threshold_grade_c_x,Year,Notes,Value,q_position,threshold_grade_a_y,threshold_grade_b_y,threshold_grade_c_y
0,11,Aceh,Province,Sumatra,SM,Average Monthly Household Consumption for Tele...,TEL.EXP.HH.PROV.CONSUM.TEL,Telecommunication Expenditure,TELCO_EXP,Telecom Exp,...,NaN,NaN,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN
1,11,Aceh,Province,Sumatra,SM,Average Monthly Household Consumption for Tele...,TEL.EXP.HH.PROV.CONSUM.TEL,Telecommunication Expenditure,TELCO_EXP,Telecom Exp,...,NaN,NaN,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN
2,11,Aceh,Province,Sumatra,SM,Average Monthly Household Consumption for Tele...,TEL.EXP.HH.PROV.CONSUM.TEL,Telecommunication Expenditure,TELCO_EXP,Telecom Exp,...,NaN,NaN,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN
3,11,Aceh,Province,Sumatra,SM,Average Monthly Household Consumption for Tele...,TEL.EXP.HH.PROV.CONSUM.TEL,Telecommunication Expenditure,TELCO_EXP,Telecom Exp,...,NaN,NaN,NaN,2021,NaN,NaN,NaN,NaN,NaN,NaN
4,11,Aceh,Province,Sumatra,SM,Average Monthly Household Consumption for Tele...,TEL.EXP.HH.PROV.CONSUM.TEL,Telecommunication Expenditure,TELCO_EXP,Telecom Exp,...,NaN,NaN,NaN,2022,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41785,00,Indonesia,Nasional,Indonesia,IDN,Percentage of Households Owning/Controlling Mo...,TEL.OWN.MOB.URB.RES.PRIV,Technology Ownership and Use,TECH_USE,Telephone,...,69.82,73.8205,77.63,2019,NaN,NaN,NaN,NaN,NaN,NaN
41786,00,Indonesia,Nasional,Indonesia,IDN,Percentage of Households Owning/Controlling Mo...,TEL.OWN.MOB.URB.RES.PRIV,Technology Ownership and Use,TECH_USE,Telephone,...,69.82,73.8205,77.63,2020,NaN,71.30,NaN,NaN,NaN,NaN
41787,00,Indonesia,Nasional,Indonesia,IDN,Percentage of Households Owning/Controlling Mo...,TEL.OWN.MOB.URB.RES.PRIV,Technology Ownership and Use,TECH_USE,Telephone,...,69.82,73.8205,77.63,2021,NaN,72.89,NaN,NaN,NaN,NaN
41788,00,Indonesia,Nasional,Indonesia,IDN,Percentage of Households Owning/Controlling Mo...,TEL.OWN.MOB.URB.RES.PRIV,Technology Ownership and Use,TECH_USE,Telephone,...,69.82,73.8205,77.63,2022,NaN,73.20,NaN,NaN,NaN,NaN


In [28]:
data_with_threshold[data_with_threshold.Value.notnull()][data_with_threshold["Area Code"] != "00"]

/tmp/ipykernel_17926/931555123.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_with_threshold[data_with_threshold.Value.notnull()][data_with_threshold["Area Code"] != "00"]


,Area Code,Area,AREA_TYPE,REGION_GROUP,ID_REGION,Indicator Name,Indicator Code,Category,Category_ID,Sub_Category,...,threshold_grade_a_x,threshold_grade_b_x,threshold_grade_c_x,Year,Notes,Value,q_position,threshold_grade_a_y,threshold_grade_b_y,threshold_grade_c_y
26,11,Aceh,Province,Sumatra,SM,Average Number of Active Mobile Phone Numbers ...,TEL.MOB.PROV.ACTNUM.AVG,Technology Ownership and Use,TECH_USE,Telephone,...,2.728,2.8460,2.954,2020,NaN,2.70,36.111111,2.954,2.8460,2.728
27,11,Aceh,Province,Sumatra,SM,Average Number of Active Mobile Phone Numbers ...,TEL.MOB.PROV.ACTNUM.AVG,Technology Ownership and Use,TECH_USE,Telephone,...,2.728,2.8460,2.954,2021,NaN,2.71,37.500000,2.954,2.8460,2.728
28,11,Aceh,Province,Sumatra,SM,Average Number of Active Mobile Phone Numbers ...,TEL.MOB.PROV.ACTNUM.AVG,Technology Ownership and Use,TECH_USE,Telephone,...,2.728,2.8460,2.954,2022,NaN,2.70,NaN,NaN,NaN,NaN
29,11,Aceh,Province,Sumatra,SM,Average Number of Active Mobile Phone Numbers ...,TEL.MOB.PROV.ACTNUM.AVG,Technology Ownership and Use,TECH_USE,Telephone,...,2.728,2.8460,2.954,2023,NaN,3.03,76.388889,2.954,2.8460,2.728
32,11,Aceh,Province,Sumatra,SM,Average Number of Household Members Owning Mob...,TEL.MOB.PROV.MEMBERS.AVG,Technology Ownership and Use,TECH_USE,Telephone,...,2.410,2.5100,2.600,2020,NaN,2.36,31.666667,2.600,2.5100,2.410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40589,94,Papua,Province,Maluku and Papua,MAP,Percentage of Households Owning/Controlling Mo...,TEL.OWN.MOB.URB.RES.FREE,Technology Ownership and Use,TECH_USE,Telephone,...,10.808,11.8130,13.134,2023,NaN,11.53,50.793651,13.134,11.8130,10.808
40592,94,Papua,Province,Maluku and Papua,MAP,Percentage of Households Owning/Controlling Mo...,TEL.OWN.MOB.URB.RES.PRIV,Technology Ownership and Use,TECH_USE,Telephone,...,69.820,73.8205,77.630,2020,NaN,55.26,3.053435,77.630,73.8205,69.820
40593,94,Papua,Province,Maluku and Papua,MAP,Percentage of Households Owning/Controlling Mo...,TEL.OWN.MOB.URB.RES.PRIV,Technology Ownership and Use,TECH_USE,Telephone,...,69.820,73.8205,77.630,2021,NaN,53.75,2.290076,77.630,73.8205,69.820
40594,94,Papua,Province,Maluku and Papua,MAP,Percentage of Households Owning/Controlling Mo...,TEL.OWN.MOB.URB.RES.PRIV,Technology Ownership and Use,TECH_USE,Telephone,...,69.820,73.8205,77.630,2022,NaN,59.57,7.633588,77.630,73.8205,69.820


In [45]:
master_indicator_df.merge(threshold_final_df.rename(columns={"Indicator Code":"Indicator_Code"}), how="left", on="Indicator_Code").to_csv("threshold.csv")

In [46]:
def write_data_to_sps(creds, sps_id:str, worksheet_name:str, df:pd.DataFrame):
    client = gspread.authorize(creds)

    spreadsheet = client.open_by_key(sps_id)
    worksheet = spreadsheet.worksheet(worksheet_name)
    worksheet.clear()
    
    area_as_a_text_format = {
        "numberFormat": {
            "type": "TEXT"
        }
    }
    num_rows = len(df) + 1
    column_range = f'A2:A{num_rows}'
    worksheet.format(column_range, area_as_a_text_format)
    
    gspread_dataframe.set_with_dataframe(worksheet=worksheet, dataframe=df)
    border_style = {
        "style": "SOLID",
        "width": 1,
    }

    formatting = {
        "borders": {
          "top": border_style,
          "bottom": border_style,
          "left": border_style,
          "right": border_style
        }
    }
    num_rows = len(df) + 1
    num_cols = len(df.columns)
    range_to_border = f"A1:{gspread.utils.rowcol_to_a1(num_rows, num_cols)}"
    worksheet.format(range_to_border, formatting)
    
    bold_format = {
        "textFormat":{
            "bold":True
        }
    }
    range_to_bold = f"A1:{gspread.utils.rowcol_to_a1(1, len(df.columns))}"
    worksheet.format(range_to_bold, bold_format)
    
    requests = [
        {
            "autoResizeDimensions": {
                "dimensions": {
                    "sheetId": worksheet.id, # The numeric ID of the worksheet
                    "dimension": "COLUMNS",
                    "startIndex": 0,
                    "endIndex": len(df.columns)
                }
            }
        }
    ]

    # Send the batch update request to the spreadsheet
    spreadsheet.batch_update(body={'requests': requests})
    
    return df

# CALL FUNCTION
# converted_data_from_sps = write_data_to_sps(creds, settings.MERGED_DATA_SPS_ID, "main",converted_value_data )
# converted_data_from_sps

## Converted Data -> Data Warehouse

In [47]:
def get_dataframe_from_sheet(creds, sps_id:str, worksheet_name:str):
    client = gspread.authorize(creds)

    spreadsheet = client.open_by_key(sps_id)
    worksheet = spreadsheet.worksheet(worksheet_name)
    df = gspread_dataframe.get_as_dataframe(worksheet=worksheet)
    
    return df
def convert_ID(row):
    row['area_code'] = str(row['area_code']).zfill(2)
    return row
#CALL FUNCTION
# converted_and_merged_data = get_dataframe_from_sheet(creds, settings.MERGED_DATA_SPS_ID, "main")
dim_area = get_dataframe_from_sheet(creds, settings.WAREHOUSE_DATA_SPS_ID, "dim_area").drop(columns="id").astype({"area_code":int}).apply(convert_ID, axis=1)
dim_indicator = get_dataframe_from_sheet(creds, settings.WAREHOUSE_DATA_SPS_ID, "dim_indicator").drop(columns="id")
dim_year = get_dataframe_from_sheet(creds, settings.WAREHOUSE_DATA_SPS_ID, "dim_year").drop(columns="id").astype({"year":int}).astype({"year":str})


/tmp/ipykernel_17926/3254538975.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  row['area_code'] = str(row['area_code']).zfill(2)


In [48]:
dim_year.columns = dim_year_column
dim_indicator.columns = dim_indicator_column
dim_area.columns = dim_area_column

In [70]:
final_dim_year = handle_dim_year(creds, dim_year, master_year_df, default_null_value)
final_dim_year

,id,year,note
0,1,2018,--EMPTY VALUE DATA--
1,2,2019,--EMPTY VALUE DATA--
2,3,2020,--EMPTY VALUE DATA--
3,4,2021,--EMPTY VALUE DATA--
4,5,2022,--EMPTY VALUE DATA--
5,6,2023,--EMPTY VALUE DATA--


In [71]:
final_dim_area = handle_dim_area(creds, dim_area, master_area_df, master_inc_province_df, default_null_value)
final_dim_area

,id,area_code,area_name,area_type,region_code,region_name,income_level_code,income_level_name
0,1,11,Aceh,Province,SM,Sumatra,LMI,Pendapatan Menengah Bawah
1,2,12,Sumatera Utara,Province,SM,Sumatra,UMI,Pendapatan Menengah Atas
2,3,13,Sumatera Barat,Province,SM,Sumatra,LMI,Pendapatan Menengah Bawah
3,4,14,Riau,Province,SM,Sumatra,UMI,Pendapatan Menengah Atas
4,5,15,Jambi,Province,SM,Sumatra,UMI,Pendapatan Menengah Atas
5,6,16,Sumatera Selatan,Province,SM,Sumatra,UMI,Pendapatan Menengah Atas
6,7,17,Bengkulu,Province,SM,Sumatra,LMI,Pendapatan Menengah Bawah
7,8,18,Lampung,Province,SM,Sumatra,LMI,Pendapatan Menengah Bawah
8,9,19,Kepulauan Bangka Belitung,Province,SM,Sumatra,UMI,Pendapatan Menengah Atas
9,10,21,Kepulauan Riau,Province,SM,Sumatra,UMI,Pendapatan Menengah Atas


In [72]:
final_dim_indicator = handle_dim_indicator(creds, dim_indicator, master_indicator_df, default_null_value)
final_dim_indicator

,id,indicator_code,indicator_name,category_id,category_name,sub_category_id,sub_category_name,indicator_area_id,indicator_area_name,unit
0,1,TEL.EXP.HH.PROV.CONSUM.TEL,Average Monthly Household Consumption for Tele...,TELCO_EXP,Telecommunication Expenditure,EXP,Telecom Exp,ALL,All,Rupiah
1,2,TEL.EXP.HH.PROV.CONSUM,Average Monthly Household Consumption with Tel...,TELCO_EXP,Telecommunication Expenditure,EXP,Telecom Exp,ALL,All,Rupiah
2,3,TEL.EXP.HH.PROV.CONSUM.FOOD,Average Monthly Household Consumption with Tel...,TELCO_EXP,Telecommunication Expenditure,EXP,Telecom Exp,ALL,All,Rupiah
3,4,TEL.EXP.HH.PROV.CONSUM.NONFOOD,Average Monthly Household Consumption with Tel...,TELCO_EXP,Telecommunication Expenditure,EXP,Telecom Exp,ALL,All,Rupiah
4,5,TEL.MOB.PROV.ACTNUM.AVG,Average Number of Active Mobile Phone Numbers ...,TECH_USE,Technology Ownership and Use,TEL,Telephone,ALL,All,Average
...,...,...,...,...,...,...,...,...,...,...
194,195,TEL.OWN.MOB.URB.RES.OFFICIAL,Percentage of Households Owning/Controlling Mo...,TECH_USE,Technology Ownership and Use,TEL,Telephone,URB,Urban,%
195,196,TEL.OWN.MOB.URB.RES.RENT,Percentage of Households Owning/Controlling Mo...,TECH_USE,Technology Ownership and Use,TEL,Telephone,URB,Urban,%
196,197,TEL.OWN.MOB.URB.RES.OTHERS,Percentage of Households Owning/Controlling Mo...,TECH_USE,Technology Ownership and Use,TEL,Telephone,URB,Urban,%
197,198,TEL.OWN.MOB.URB.RES.FREE,Percentage of Households Owning/Controlling Mo...,TECH_USE,Technology Ownership and Use,TEL,Telephone,URB,Urban,%


In [69]:


final_fact_digital_equity = handle_fact_table(creds,
                                              settings.WAREHOUSE_DATA_SPS_ID,
                                              "fact_digital_equity",
                                              data_with_threshold[["Indicator Code", "Area Code", "Year", "Value", "q_position"]],
                                              final_dim_year,
                                              final_dim_area,
                                              final_dim_indicator)
final_fact_digital_equity


,dim_year_id,dim_indicator_id,dim_area_id,value,grade_index
26,3,5,1,2.70,31.111111
27,4,5,1,2.71,36.296296
28,5,5,1,2.70,31.111111
29,6,5,1,3.03,83.703704
32,3,6,1,2.36,27.407407
...,...,...,...,...,...
41783,6,198,35,13.18,NaN
41786,3,199,35,71.30,NaN
41787,4,199,35,72.89,NaN
41788,5,199,35,73.20,NaN


In [ ]:
source_fact = data_with_threshold[["Indicator Code", "Area Code", "Year", "Value", "q_position"]]
source_fact.columns = ["indicator_code", "area_code", "year", "value", "grade_index"]
enriched_df = source_fact.merge(final_dim_year[["id", "year"]], how="inner", on="year").rename(columns={"id":"dim_year_id"})
enriched_df = enriched_df.merge(final_dim_area[["id", "area_code"]], how="inner", on="area_code").rename(columns={"id":"dim_area_id"})
enriched_df = enriched_df.merge(final_dim_indicator[["id", 
                                                     "indicator_code"]], 
                                how="inner", 
                                on="indicator_code").rename(columns={"id":"dim_indicator_id"})

final_fact_digital_equity = enriched_df[enriched_df.value.notnull()][fact_digital_equity]
# write_data_to_sps(creds, settings.WAREHOUSE_DATA_SPS_ID, "fact_digital_equity", final_fact_digital_equity)
final_fact_digital_equity


,dim_year_id,dim_indicator_id,dim_area_id,value,grade_index
26,3,5,1,2.70,31.111111
27,4,5,1,2.71,36.296296
28,5,5,1,2.70,31.111111
29,6,5,1,3.03,83.703704
32,3,6,1,2.36,27.407407
...,...,...,...,...,...
41783,6,198,35,13.18,NaN
41786,3,199,35,71.30,NaN
41787,4,199,35,72.89,NaN
41788,5,199,35,73.20,NaN


,dim_year_id,dim_indicator_id,dim_area_id,value,grade_index
26,3,5,1,2.70,31.111111
27,4,5,1,2.71,36.296296
28,5,5,1,2.70,31.111111
29,6,5,1,3.03,83.703704
32,3,6,1,2.36,27.407407
...,...,...,...,...,...
40589,6,198,34,11.53,48.888889
40592,3,199,34,55.26,2.962963
40593,4,199,34,53.75,2.222222
40594,5,199,34,59.57,7.407407


In [73]:
source_fact = data_with_threshold[["Indicator_Code", "ID", "Year", "Value", "q_position"]]
source_fact.columns = ["indicator_code", "area_code", "year", "value", "grade_index"]

enriched_df = source_fact.merge(final_dim_year[["id", "year"]], how="inner", on="year").rename(columns={"id":"dim_year_id"})
enriched_df = enriched_df.merge(final_dim_area[["id", "area_code"]], how="inner", on="area_code").rename(columns={"id":"dim_area_id"})
enriched_df = enriched_df.merge(final_dim_indicator[["id", 
                                                     "indicator_code",
                                                     "threshold_grade_a",
                                                     "threshold_grade_b",
                                                     "threshold_grade_c"]], 
                                how="inner", 
                                on="indicator_code").rename(columns={"id":"dim_indicator_id"})

def set_grade(value,
              tgA,
              tgB,
              tgC,
              area_code):
        if math.isnan(value) or area_code == "00": return np.nan
        if value > tgA:
            return "A"
        elif value > tgB:
            return "B"
        elif value > tgC:
            return "C"
        else:
            return "D"

enriched_df['grade'] = enriched_df.apply(lambda row : set_grade(row['value'],
                                         row['threshold_grade_a'],
                                         row['threshold_grade_b'],
                                         row['threshold_grade_c'],
                                         row['area_code']), axis=1)

fact_table = enriched_df[fact_it_eco_column]
# fact_table = fact_table[fact_table.value.]
write_data_to_sps(creds, settings.WAREHOUSE_DATA_SPS_ID, "fact_it_ecosystem", fact_table)
# fact_table

,dim_year_id,dim_indicator_id,dim_area_id,value,grade,grade_index
0,1,1,1,2987742.00,B,55.172414
1,2,1,1,3398557.00,B,65.517241
2,3,1,1,2422040.00,D,39.901478
3,4,1,1,2691729.00,C,46.798030
4,5,1,1,198263.00,D,0.985222
...,...,...,...,...,...,...
42173,2,198,35,56.60,NaN,NaN
42174,3,198,35,54.65,NaN,NaN
42175,4,198,35,92.11,NaN,NaN
42176,5,198,35,52.00,NaN,NaN
